# Adaptive

[`adaptive`](https://gitlab.kwant-project.org/qt/adaptive-evaluation) is a package for adaptively sampling functions with support for parallel evaluation.

This is an introductory notebook that shows some basic use cases.

`adaptive` needs at least Python 3.6, and the following packages:

+ `scipy`
+ `sortedcontainers`

Additionally `adaptive` has lots of extra functionality that makes it simple to use from Jupyter notebooks.
This extra functionality depends on the following packages

+ `ipykernel>=4.8.0`
+ `jupyter_client>=5.2.2`
+ `holoviews`
+ `bokeh`
+ `ipywidgets`

In [ ]:
import adaptive
adaptive.notebook_extension()

# Import modules that are used in multiple cells
import holoviews as hv
import numpy as np
from functools import partial
import random

# 1D function learner

We start with the most common use-case: sampling a 1D function $\ f: ℝ → ℝ$.

We will use the following function, which is a smooth (linear) background with a sharp peak at a random location:

In [ ]:
offset = random.uniform(-0.5, 0.5)

def f(x, offset=offset, wait=True):
    from time import sleep
    from random import random

    a = 0.01
    if wait:
        sleep(random())
    return x + a**2 / (a**2 + (x - offset)**2)

We start by initializing a 1D "learner", which will suggest points to evaluate, and adapt its suggestions as more and more points are evaluated.

In [ ]:
learner = adaptive.Learner1D(f, bounds=(-1, 1))

Next we create a "runner" that will request points from the learner and evaluate 'f' on them.

By default the runner will evaluate the points in parallel using local processes ([`concurrent.futures.ProcessPoolExecutor`](https://docs.python.org/3/library/concurrent.futures.html#processpoolexecutor)).

In [ ]:
# The end condition is when the "loss" is less than 0.1. In the context of the
# 1D learner this means that we will resolve features in 'func' with width 0.1 or wider.
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.05)
runner.live_info()

When instantiated in a Jupyter notebook the runner does its job in the background and does not block the IPython kernel.
We can use this to create a plot that updates as new data arrives:

In [ ]:
runner.live_plot(update_interval=0.1)

We can now compare the adaptive sampling to a homogeneous sampling with the same number of points:

In [ ]:
if not runner.task.done():
    raise RuntimeError('Wait for the runner to finish before executing the cells below!')

In [ ]:
learner2 = adaptive.Learner1D(f, bounds=learner.bounds)

xs = np.linspace(*learner.bounds, len(learner.data))
learner2.add_data(xs, map(partial(f, wait=False), xs))

learner.plot() + learner2.plot()

# 2D function learner

Besides 1D functions, we can also learn 2D functions: $\ f: ℝ^2 → ℝ$

In [ ]:
def ring(xy, wait=True):
    import numpy as np
    from time import sleep
    from random import random
    if wait:
        sleep(random()/10)
    x, y = xy
    a = 0.2
    return x + np.exp(-(x**2 + y**2 - 0.75**2)**2/a**4)

learner = adaptive.Learner2D(ring, bounds=[(-1, 1), (-1, 1)])

In [ ]:
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.01)
runner.live_info()

In [ ]:
def plot(learner):
    plot = learner.plot(tri_alpha=0.2)
    title = f'loss={learner._loss:.3f}, n_points={learner.npoints}'
    return (plot.Image
            + plot.EdgePaths.I.opts(plot=dict(title_format=title))
            + plot)

runner.live_plot(plotter=plot, update_interval=0.1)

In [ ]:
%%opts EdgePaths (color='w')

import itertools

# Create a learner and add data on homogeneous grid, so that we can plot it
learner2 = adaptive.Learner2D(ring, bounds=learner.bounds)
n = int(learner.npoints**0.5)
xs, ys = [np.linspace(*bounds, n) for bounds in learner.bounds]
xys = list(itertools.product(xs, ys))
learner2.add_data(xys, map(partial(ring, wait=False), xys))

(learner2.plot(n).relabel('Homogeneous grid') + learner.plot().relabel('With adaptive') + 
 learner2.plot(n, tri_alpha=0.4) + learner.plot(tri_alpha=0.4)).cols(2)

# Averaging learner

The next type of learner averages a function until the uncertainty in the average meets some condition.

This is useful for sampling a random variable. The function passed to the learner must formally take a single parameter,
which should be used like a "seed" for the (pseudo-) random variable (although in the current implementation the seed parameter can be ignored by the function).

In [ ]:
def g(n):
    import random
    from time import sleep
    sleep(random.random() / 1000)
    # Properly save and restore the RNG state
    state = random.getstate()
    random.seed(n)
    val = random.gauss(0.5, 1)
    random.setstate(state)
    return val

In [ ]:
learner = adaptive.AverageLearner(g, atol=None, rtol=0.01)
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 2)
runner.live_info()

In [ ]:
runner.live_plot(update_interval=0.1)

# 1D integration learner with `cquad`

This learner learns a 1D function and calculates the integral and error of the integral with it. It is based on Pedro Gonnet's [implementation](https://www.academia.edu/1976055/Adaptive_quadrature_re-revisited).

Let's try the following function with cusps (that is difficult to integrate):

In [ ]:
def f24(x):
    return np.floor(np.exp(x))

xs = np.linspace(0, 3, 200)
hv.Scatter((xs, [f24(x) for x in xs]))

Just to prove that this really is a difficult to integrate function, let's try a familiar function integrator `scipy.integrate.quad`, which will give us warnings that it encounters difficulties.

In [ ]:
import scipy.integrate
scipy.integrate.quad(f24, 0, 3)

We initialize a learner again and pass the bounds and relative tolerance we want to reach. Then in the `Runner` we pass `goal=lambda l: l.done()` where `learner.done()` is `True` when the relative tolerance has been reached.

In [ ]:
from adaptive.runner import SequentialExecutor

learner = adaptive.IntegratorLearner(f24, bounds=(0, 3), tol=1e-10)

# We use a SequentialExecutor, which runs the function to be learned in *this* process only. This means we don't pay
# the overhead of evaluating the function in another process.
runner = adaptive.Runner(learner, executor=SequentialExecutor(), goal=lambda l: l.done())
runner.live_info()

Now we could do the live plotting again, but lets just wait untill the runner is done.

In [ ]:
if not runner.task.done():
    raise RuntimeError('Wait for the runner to finish before executing the cells below!')

In [ ]:
print('The integral value is {} with the corresponding error of {}'.format(learner.igral, learner.err))
learner.plot()

# 1D learner with vector output: `f:ℝ → ℝ^N`

Sometimes you may want to learn a function with vector output:

In [ ]:
random.seed(0)
offsets = [random.uniform(-0.8, 0.8) for _ in range(3)]

# sharp peaks at random locations in the domain
def f_levels(x, offsets=offsets):
    a = 0.01
    return np.array([offset + x + a**2 / (a**2 + (x - offset)**2)
                     for offset in offsets])

`adaptive` has you covered! The `Learner1D` can be used for such functions:

In [ ]:
learner = adaptive.Learner1D(f_levels, bounds=(-1, 1))
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.05)
runner.live_plot()

# Custom point choosing logic for 1D and 2D

The `Learner1D` and `Learner2D` implement a certain logic for chosing points based on the existing data.

For some functions this default stratagy might not work, for example you'll run into trouble when you learn functions that contain divergencies.

Both the `Learner1D` and `Learner2D` allow you to use a custom loss function, which you specify as an argument in the learner. See the doc-string of `Learner1D` and `Learner2D` to see what `loss_per_interval` and `loss_per_triangle` need to return and take as input.

As an example we implement a homogeneous sampling strategy (which of course is not the best way of handling divergencies).

Note that both these loss functions are also available from `adaptive.learner.learner1d.uniform_sampling` and `adaptive.learner.learner2d.uniform_sampling`.

In [ ]:
def uniform_sampling_1d(interval, scale, function_values):
    x_left, x_right = interval
    x_scale, _ = scale
    dx = (x_right - x_left) / x_scale
    return dx

def f_divergent_1d(x):
    return 1 / x**2

learner = adaptive.Learner1D(f_divergent_1d, (-1, 1), loss_per_interval=uniform_sampling_1d)
runner = adaptive.BlockingRunner(learner, goal=lambda l: l.loss() < 0.01)
learner.plot().select(y=(0, 10000))

In [ ]:
%%opts EdgePaths (color='w')

def uniform_sampling_2d(ip):
    from adaptive.learner.learner2D import areas
    A = areas(ip)
    return np.sqrt(A)

def f_divergent_2d(xy):
    x, y = xy
    return 1 / (x**2 + y**2)

learner = adaptive.Learner2D(f_divergent_2d, [(-1, 1), (-1, 1)], loss_per_triangle=uniform_sampling_2d)
runner = adaptive.BlockingRunner(learner, goal=lambda l: l.loss() < 0.02)
learner.plot(tri_alpha=0.3)

#### Doing better
Of course we can improve on the the above result, since just homogeneous sampling is usually the dumbest way to sample.

In [ ]:
%%opts EdgePaths (color='w') Image [logz=True]

def max_resolution_loss(ip, smallest_distance=0.01):
    from adaptive.learner.learner2D import areas, deviations
    A = areas(ip)

    # `deviations` returns an array of the same length as the
    # vector your function to be learned returns, so 1 in this case.
    # Its value represents the deviation from the linear estimate based
    # on the gradients inside each triangle.
    dev = deviations(ip)[0]
    
    # we add terms of the same dimension: dev == [distance], A == [distance**2]
    loss = np.sqrt(A) * dev + A
    
    # Setting areas with a small area to zero such that they won't be chosen again
    loss[A < smallest_distance**2] = 0 
    return loss

learner = adaptive.Learner2D(f_divergent_2d, [(-1, 1), (-1, 1)], loss_per_triangle=max_resolution_loss)
runner = adaptive.BlockingRunner(learner, goal=lambda l: l.loss() < 0.02)
learner.plot(tri_alpha=0.3).relabel('Plotted in log scale')

# Balancing learner

The balancing learner is a "meta-learner" that takes a list of learners. When you request a point from the balancing learner, it will query all of its "children" to figure out which one will give the most improvement.

The balancing learner can for example be used to implement a poor-man's 2D learner by using the `Learner1D`.

In [ ]:
def f(x, offset=0):
    a = 0.01
    return x + a**2 / (a**2 + (x - offset)**2)

learners = [adaptive.Learner1D(partial(f, offset=random.uniform(-1, 1)),
            bounds=(-1, 1)) for i in range(10)]

bal_learner = adaptive.BalancingLearner(learners)
runner = adaptive.Runner(bal_learner, goal=lambda l: l.loss() < 0.01)
runner.live_info()

In [ ]:
plotter = lambda learner: hv.Overlay([L.plot() for L in learner.learners])
runner.live_plot(plotter=plotter, update_interval=0.1)

# DataSaver

If the function that you want to learn returns a value along with some metadata, you can wrap your learner in an `adaptive.DataSaver`.

In the following example the function to be learned returns its result and the execution time in a dictionary:

In [ ]:
from operator import itemgetter

def f_dict(x):
    """The function evaluation takes roughly the time we `sleep`."""
    import random
    from time import sleep

    waiting_time = random.random()
    sleep(waiting_time)
    a = 0.01
    y = x + a**2 / (a**2 + x**2)
    return {'y': y, 'waiting_time': waiting_time}

# Create the learner with the function that returns a 'dict'
# This learner cannot be run directly, as Learner1D does not know what to do with the 'dict'
_learner = adaptive.Learner1D(f_dict, bounds=(-1, 1))

# Wrapping the learner with 'adaptive.DataSaver' and tell it which key it needs to learn
learner = adaptive.DataSaver(_learner, arg_picker=itemgetter('y'))

`learner.learner` is the original learner, so `learner.learner.loss()` will call the correct loss method.

In [ ]:
runner = adaptive.Runner(learner, goal=lambda l: l.learner.loss() < 0.05)
runner.live_info()

In [ ]:
runner.live_plot(plotter=lambda l: l.learner.plot(), update_interval=0.1)

Now the `DataSavingLearner` will have an dictionary attribute `extra_data` that has `x` as key and the data that was returned by `learner.function` as values.

In [ ]:
learner.extra_data

# Using multiple cores

Often you will want to evaluate the function on some remote computing resources. `adaptive` works out of the box with any framework that implements a [PEP 3148](https://www.python.org/dev/peps/pep-3148/) compliant executor that returns `concurrent.futures.Future` objects.

### [`concurrent.futures`](https://docs.python.org/3/library/concurrent.futures.html)

By default `adaptive.Runner` creates a `ProcessPoolExecutor`, but you can also pass one explicitly e.g. to limit the number of workers:

In [ ]:
from concurrent.futures import ProcessPoolExecutor

executor = ProcessPoolExecutor(max_workers=4)

learner = adaptive.Learner1D(f, bounds=(-1, 1))
runner = adaptive.Runner(learner, executor=executor, goal=lambda l: l.loss() < 0.05)
runner.live_info()
runner.live_plot(update_interval=0.1)

### [`ipyparallel`](https://ipyparallel.readthedocs.io/en/latest/intro.html)

In [ ]:
import ipyparallel

client = ipyparallel.Client()  # You will need to start an `ipcluster` to make this work

learner = adaptive.Learner1D(f, bounds=(-1, 1))
runner = adaptive.Runner(learner, executor=client, goal=lambda l: l.loss() < 0.01)
runner.live_info()
runner.live_plot()

### [`distributed`](https://distributed.readthedocs.io/en/latest/)

In [ ]:
import distributed

client = distributed.Client()

learner = adaptive.Learner1D(f, bounds=(-1, 1))
runner = adaptive.Runner(learner, executor=client, goal=lambda l: l.loss() < 0.01)
runner.live_info()
runner.live_plot(update_interval=0.1)

---

# Advanced Topics

## A watched pot never boils!

`adaptive.Runner` does its work in an `asyncio` task that runs concurrently with the IPython kernel, when using `adaptive` from a Jupyter notebook. This is advantageous because it allows us to do things like live-updating plots, however it can trip you up if you're not careful.

Notably: **if you block the IPython kernel, the runner will not do any work**.

For example if you wanted to wait for a runner to complete, **do not wait in a busy loop**:
```python
while not runner.task.done():
    pass
```

If you do this then **the runner will never finish**.

What to do if you don't care about live plotting, and just want to run something until its done?

The simplest way to accomplish this is to use `adaptive.BlockingRunner`:

In [ ]:
learner = adaptive.Learner1D(f, bounds=(-1, 1))
adaptive.BlockingRunner(learner, goal=lambda l: l.loss() < 0.005)
# This will only get run after the runner has finished
learner.plot()

## Reproducibility

By default `adaptive` runners evaluate the learned function in parallel across several cores. The runners are also opportunistic, in that as soon as a result is available they will feed it to the learner and request another point to replace the one that just finished.

Because the order in which computations complete is non-deterministic, this means that the runner behaves in a non-deterministic way. Adaptive makes this choice because in many cases the speedup from parallel execution is worth sacrificing the "purity" of exactly reproducible computations.

Nevertheless it is still possible to run a learner in a deterministic way with adaptive.

The simplest way is to use `adaptive.runner.simple` to run your learner:

In [ ]:
learner = adaptive.Learner1D(f, bounds=(-1, 1))

# blocks until completion
adaptive.runner.simple(learner, goal=lambda l: l.loss() < 0.002)

learner.plot()

Note that unlike `adaptive.Runner`, `adaptive.runner.simple` *blocks* until it is finished.

If you want to enable determinism, want to continue using the non-blocking `adaptive.Runner`, you can use the `adaptive.runner.SequentialExecutor`:

In [ ]:
from adaptive.runner import SequentialExecutor

learner = adaptive.Learner1D(f, bounds=(-1, 1))

# blocks until completion
runner = adaptive.Runner(learner, executor=SequentialExecutor(), goal=lambda l: l.loss() < 0.002)
runner.live_info()
runner.live_plot(update_interval=0.1)

## Cancelling a runner

Sometimes you want to interactively explore a parameter space, and want the function to be evaluated at finer and finer resolution and manually control when the calculation stops.

If no `goal` is provided to a runner then the runner will run until cancelled.

`runner.live_info()` will provide a button that can be clicked to stop the runner. You can also stop the runner programatically using `runner.cancel()`.

In [ ]:
learner = adaptive.Learner1D(f, bounds=(-1, 1))
runner = adaptive.Runner(learner)
runner.live_info()
runner.live_plot(update_interval=0.1)

In [ ]:
runner.cancel()

In [ ]:
print(runner.status())

## Debugging Problems 

Runners work in the background with respect to the IPython kernel, which makes it convenient, but also means that inspecting errors is more difficult because exceptions will not be raised directly in the notebook. Often the only indication you will have that something has gone wrong is that nothing will be happening.

Let's look at the following example, where the function to be learned will raise an exception 10% of the time.

In [ ]:
def will_raise(x):
    from random import random
    from time import sleep
    
    sleep(random())
    if random() < 0.1:
        raise RuntimeError('something went wrong!')
    return x**2
    
learner = adaptive.Learner1D(will_raise, (-1, 1))
runner = adaptive.Runner(learner)  # without 'goal' the runner will run forever unless cancelled
runner.live_info()
runner.live_plot()

The above runner should continue forever, but we notice that it stops after a few points are evaluated.

First we should check that the runner has really finished:

In [ ]:
runner.task.done()

If it has indeed finished then we should check the `result` of the runner. This should be `None` if the runner stopped successfully. If the runner stopped due to an exception then asking for the result will raise the exception with the stack trace:

In [ ]:
runner.task.result()

### Logging runners

Runners do their job in the background, which makes introspection quite cumbersome. One way to inspect runners is to instantiate one with `log=True`:

In [ ]:
learner = adaptive.Learner1D(f, bounds=(-1, 1))
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.1,
                         log=True)
runner.live_info()

This gives a the runner a `log` attribute, which is a list of the `learner` methods that were called, as well as their arguments. This is useful because executors typically execute their tasks in a non-deterministic order.

This can be used with `adaptive.runner.replay_log` to perfom the same set of operations on another runner:


In [ ]:
reconstructed_learner = adaptive.Learner1D(f, bounds=learner.bounds)
adaptive.runner.replay_log(reconstructed_learner, runner.log)

In [ ]:
learner.plot().opts(style=dict(size=6)) * reconstructed_learner.plot()

### Timing functions

To time the runner you **cannot** simply use 
```python
now = datetime.now()
runner = adaptive.Runner(...)
print(datetime.now() - now)
```
because this will be done immediately. Also blocking the kernel with `while not runner.task.done()` will not work because the runner will not do anything when the kernel is blocked.

Therefore you need to create an `async` function and hook it into the `ioloop` like so:

In [ ]:
import asyncio

async def time(runner):
    from datetime import datetime
    now = datetime.now()
    await runner.task
    return datetime.now() - now

ioloop = asyncio.get_event_loop()

learner = adaptive.Learner1D(f, bounds=(-1, 1))
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.01)

timer = ioloop.create_task(time(runner))

In [ ]:
# The result will only be set when the runner is done.
timer.result()

## Using Runners from a script 

Runners can also be used from a Python script independently of the notebook.

The simplest way to accomplish this is simply to use the `BlockingRunner`:

```python
import adaptive

def f(x):
    return x

learner = adaptive.Learner1D(f, (-1, 1))

adaptive.BlockingRunner(learner, goal=lambda: l: l.loss() < 0.1)
```

If you use `asyncio` already in your script and want to integrate `adaptive` into it, then you can use the default `Runner` as you would from a notebook. If you want to wait for the runner to finish, then you can simply
```python
    await runner.task
```
from within a coroutine.